In [2]:
import json
import os
import urllib.request

import petl

In [5]:
stations_table = petl.fromcsv('raw/2015_station_data.csv')

In [6]:
json_elevations_filename = 'elevations.json'
csv_elevations_filename = 'elevations.csv'

# Get station elevations via Google Maps API. use of the API inspired by https://jakevdp.github.io/blog/2015/10/17/analyzing-pronto-cycleshare-data-with-python-and-pandas/
url = "https://maps.googleapis.com/maps/api/elevation/json?locations=" + \
    '|'.join([
        '{0},{1}'.format(record.lat, record.long)
        for record in stations_table.records()
    ])
print('fetching elevation data from Google API at', url)
urllib.request.urlretrieve(url, json_elevations_filename)

with open(json_elevations_filename) as json_elevations_file:
    lat_lon_elevation_table = petl.fromdicts([
        {'elevation': result['elevation'], 'lat': result['location']['lat'], 'lon': result['location']['lng']}
        for result in json.load(json_elevations_file)['results']
    ])
lat_lon_elevation_table.tocsv(csv_elevations_filename)
lat_lon_elevation_table

fetching elevation data from Google API at https://maps.googleapis.com/maps/api/elevation/json?locations=47.618418,-122.350964|47.615829,-122.348564|47.616094,-122.341102|47.61311,-122.344208|47.610185,-122.339641|47.610731,-122.332447|47.603509,-122.330409|47.613715,-122.331777|47.60595,-122.335768|47.618633,-122.325249|47.622063,-122.321251|47.623367,-122.325279|47.620712,-122.312805|47.618549,-122.317017|47.61533,-122.311752|47.615486,-122.318245|47.615517,-122.322083|47.615456,-122.326729|47.624142,-122.316811|47.624298,-122.339617|47.663509,-122.284119|47.627643,-122.332576|47.634831,-122.326634|47.64909,-122.322983|47.607281,-122.324783|47.609239,-122.316651|47.598488,-122.326412|47.598994,-122.329684|47.600757,-122.332946|47.619859,-122.330304|47.627735,-122.342232|47.623165,-122.338203|47.61832,-122.338913|47.613628,-122.337341|47.626041,-122.335831|47.618285,-122.342205|47.622277,-122.35523|47.666145,-122.301491|47.658288,-122.313334|47.656395,-122.31562|47.663143,-122.315086|

elevation,lat,lon
37.35177993774414,47.618418,-122.350964
33.81583023071289,47.615829,-122.348564
34.27405548095703,47.616094,-122.341102
44.28325653076172,47.61311,-122.344208
42.46038055419922,47.610185,-122.339641


In [8]:
# use annex for a quick join, assuming that the rows are in the same order
joined_table = stations_table.annex(lat_lon_elevation_table.rename({'lat': 'google_lat', 'lon': 'google_long'}))
# Row order should have been the same coming back from the Googles, but let's just check
for record in joined_table.records():
    if str(record.lat) != str(record.google_lat) or str(record.long) != str(record.google_long):
        raise Exception('this row is broken: ', record)

joined_table

id,name,terminal,lat,long,dockcount,online,elevation,google_lat,google_long
1,3rd Ave & Broad St,BT-01,47.618418,-122.350964,18,10/13/2014,37.35177993774414,47.618418,-122.350964
2,2nd Ave & Vine St,BT-03,47.615829,-122.348564,16,10/13/2014,33.81583023071289,47.615829,-122.348564
3,6th Ave & Blanchard St,BT-04,47.616094,-122.341102,16,10/13/2014,34.27405548095703,47.616094,-122.341102
4,2nd Ave & Blanchard St,BT-05,47.61311,-122.344208,14,10/13/2014,44.28325653076172,47.61311,-122.344208
5,2nd Ave & Pine St,CBD-13,47.610185,-122.339641,18,10/13/2014,42.46038055419922,47.610185,-122.339641


In [11]:
try:
    os.mkdir('csv')
except Exception:
    # probably the directory already exists
    pass
joined_table.tocsv('csv/stations_with_elevation.csv')
joined_table.tojson('json/stations_with_elevation.json')